In [14]:

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline, make_pipeline
#from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, r2_score
from sklearn.impute import SimpleImputer
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor, VotingRegressor, BaggingRegressor
# from sklearn.ensemble import VotingRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.ensemble import HistGradientBoostingRegressor
import numpy as np

# review = pd.read_excel('C:/Users/leoba/tablea.xlsx',sheet_name='Planilha4')


# import pandas as pd

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score

# Carrega o DataFrame
review = pd.read_excel('C:/Users/leoba/tablea.xlsx', sheet_name='Planilha4').drop(columns=['codigo'])

# Inicializa dicionário para armazenar os scores
r2_scores = {}

# Loop sobre todas as colunas
for target_col in review.columns:
    try:
        # Define y como a coluna atual
        y = review[target_col].astype(float)

        # Cria X removendo a coluna alvo
        X = review.drop(columns=[target_col])

        # Remove linhas com NaN em y
        mask = ~np.isnan(y)
        X = X[mask]
        y = y[mask]

        # Conta valores não nulos por coluna
        valid_counts = X.count()

        # Mantém apenas colunas com pelo menos 3 valores válidos
        X = X.loc[:, valid_counts >= 4]

        if len(y) < 4:
            continue

        else:
            test_size = 0.30
    
            X_train_val, X_test, y_train_val, y_test = train_test_split(
                X, y, test_size=test_size, random_state=42)
            X_train, X_val, y_train, y_val = train_test_split(
                X_train_val, y_train_val, test_size=test_size, random_state=42)
    
            
            
            rnd_clf = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, random_state=42)
            # rnd_clf.fit(X_train, y_train)
            
            
            bag_clf = BaggingRegressor(
                DecisionTreeRegressor(max_features="sqrt", max_leaf_nodes=16),
                n_estimators=500, random_state=42)
            
            hist_clf = HistGradientBoostingRegressor(max_iter=200, max_depth=10, random_state=42)
            
            # random_forest_clf = RandomForestRegressor(n_estimators=100, random_state=42)
            extra_trees_clf = ExtraTreesRegressor(n_estimators=100, max_depth=10, random_state=42)
            
            
            voting_clf = VotingRegressor(
                estimators=[('bag', bag_clf), ('rf', rnd_clf), ('hist', hist_clf),('extr', extra_trees_clf)])
            #voting_clf.fit(X_train, y_train)
            
            
            pipeline = Pipeline([
                ('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer(strategy='mean')),  # Você pode escolher 'median', 'most_frequent' etc.
                ('model', RandomForestRegressor(random_state=42))])
            
            param_distribs = [
                {
                    'model': [RandomForestRegressor(random_state=42)],
                    'model__n_estimators': randint(100, 1000),
                    'model__max_depth': randint(2, 20),
                    'model__max_features': randint(2, 20)
                },
                {
                    'model': [ExtraTreesRegressor(random_state=42)],
                    'model__n_estimators': randint(100, 1000),
                    'model__max_depth': randint(2, 20),
                    'model__max_features': randint(2, 20)
                },
                {
                    'model': [HistGradientBoostingRegressor(random_state=42)],
                    'model__max_iter': randint(100, 500),
                    'model__max_depth': randint(2, 20),
                    'model__learning_rate': [0.01, 0.05, 0.1]
                },
                {
            
                    'model': [BaggingRegressor(DecisionTreeRegressor(max_features="sqrt", max_leaf_nodes=16), n_estimators=500, random_state=42)]
                    
                },
                {
                    'model': [voting_clf]  # VotingRegressor já configurado
                }
            ]
            
            # RandomizedSearchCV para testar todos os modelos
            rnd_search = RandomizedSearchCV(
                pipeline,
                param_distributions=param_distribs,
                n_iter=10,
                cv=3,
                scoring='r2',  # ou 'neg_mean_squared_error' para regressão
                random_state=42,
                verbose=1
            )
            
            rnd_search.fit(X_train, y_train)
            
            
            best_model = rnd_search.best_estimator_
            y_pred = best_model.predict(X_test)
            # # Treina o modelo
            # model = RandomForestRegressor(random_state=42)
            # model.fit(X_train, y_train)
    
            # # Faz a predição e calcula o R²
            # y_pred = model.predict(X_test)
            score = r2_score(y_test, y_pred)
    
            # Armazena o resultado
            r2_scores[target_col] = score
            print(best_model)
    except Exception as e:
        r2_scores[target_col] = f"Erro: {e}"
    
    # Exibe os resultados ordenados
    #sorted_scores = sorted(r2_scores.items(), key=lambda x: x[1] if isinstance(x[1], float) else -999, reverse=True)
for col, score in r2_scores:
    print(f"{col}: {score}")



Fitting 3 folds for each of 10 candidates, totalling 30 fits
Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 BaggingRegressor(estimator=DecisionTreeRegressor(max_features='sqrt',
                                                                  max_leaf_nodes=16),
                                  n_estimators=500, random_state=42))])
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 HistGradientBoostingRegressor(learning_rate=0.01, max_depth=8,
                                               max_iter=221,
                                               random_state=42))])
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Pipeline(steps=[('preprocessing', StandardScaler()),


C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\impute\_base.py:637: UserWarn

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 BaggingRegressor(estimator=DecisionTreeRegressor(max_features='sqrt',
                                                                  max_leaf_nodes=16),
                                  n_estimators=500, random_state=42))])
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 HistGradientBoostingRegressor(learning_rate=0.01, max_depth=8,
                                               max_iter=221,
                                               random_state=42))])
Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\impute\_base.py:637: UserWarn

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 HistGradientBoostingRegressor(learning_rate=0.01, max_depth=8,
                                               max_iter=221,
                                               random_state=42))])
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 ExtraTreesRegressor(max_depth=2, max_features=13,
                                     n_estimators=413, random_state=42))])
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 BaggingRegressor(estimator=DecisionTreeRegressor(max_features='sqrt',
                                                                  

...

C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\impute\_base.py:637: UserWarn

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 ExtraTreesRegressor(max_depth=2, max_features=13,
                                     n_estimators=413, random_state=42))])
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 HistGradientBoostingRegressor(learning_rate=0.01, max_depth=8,
                                               max_iter=221,
                                               random_state=42))])
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 ExtraTreesRegressor(max_depth=2, max_features=13,
                                     n_estimators=413, random_state=42))])
Fitting 3 f

C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
C:\Users\leoba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\impute\_base.py:637: UserWarn

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 BaggingRegressor(estimator=DecisionTreeRegressor(max_features='sqrt',
                                                                  max_leaf_nodes=16),
                                  n_estimators=500, random_state=42))])
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Pipeline(steps=[('preprocessing', StandardScaler()),
                ('imputer', SimpleImputer()),
                ('model',
                 ExtraTreesRegressor(max_depth=2, max_features=13,
                                     n_estimators=413, random_state=42))])
A: l
A: s


ValueError: too many values to unpack (expected 2)

In [ ]:
##### COMO GUARDAR OS MODELOS
import joblib

joblib.dump(best_model, "_model.pkl")

final_model_reloaded = joblib.load("_model.pkl")